In [158]:
%load_ext autoreload
%autoreload 2

demo
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
from pydantic import BaseModel, field_validator, Field

## JSON Schema specific limitations
type contraints: no generics, no functions
no custom validation function
no native support for:


## Basic pydantic models

In [160]:
class StringSimple(BaseModel):
    username: str
    email: str
    

print(StringSimple.model_json_schema())

{'properties': {'username': {'title': 'Username', 'type': 'string'}, 'email': {'title': 'Email', 'type': 'string'}}, 'required': ['username', 'email'], 'title': 'StringSimple', 'type': 'object'}


In [161]:
from ipywidgets_jsonschema import Form
def show(model):
    form = Form(model.model_json_schema())
    form.show()



## Nested Model

In [162]:
from typing import List
class Address(BaseModel):
    street: str
    city: str
class StringNested(BaseModel):
    name: str
    adresses: List[Address]
print(StringNested.model_json_schema())
show(StringNested)

{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}}, 'required': ['street', 'city'], 'title': 'Address', 'type': 'object'}}, 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'adresses': {'items': {'$ref': '#/$defs/Address'}, 'title': 'Adresses', 'type': 'array'}}, 'required': ['name', 'adresses'], 'title': 'StringNested', 'type': 'object'}


In [163]:
class JSONContainer:
    def __init__(self, data):
        self._data = data
    def _repr_json_(self):
        return self._data

In [164]:
JSONContainer(StringNested.model_json_schema())

In [165]:
class StringAndInt(BaseModel):
    name: str
    age: int
print(StringAndInt.model_json_schema())

{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}}, 'required': ['name', 'age'], 'title': 'StringAndInt', 'type': 'object'}


In [166]:
from pydantic import Field

class FloatMultipleOfMinumumContraint(BaseModel):
    price: float = Field(..., multiple_of = 0.5, minimum = 0.0)


print(FloatMultipleOfMinumumContraint.model_json_schema())
show(FloatMultipleOfMinumumContraint)

{'properties': {'price': {'minimum': 0.0, 'multipleOf': 0.5, 'title': 'Price', 'type': 'number'}}, 'required': ['price'], 'title': 'FloatMultipleOfMinumumContraint', 'type': 'object'}


In [167]:
from pydantic import Field
class StringMinLengthMaxLengthContraint(BaseModel):
    username: str = Field(..., min_length = 3, max_length=30)
    email: str = Field(..., format='email')
print(StringMinLengthMaxLengthContraint.model_json_schema())
show(StringMinLengthMaxLengthContraint)

{'properties': {'username': {'maxLength': 30, 'minLength': 3, 'title': 'Username', 'type': 'string'}, 'email': {'format': 'email', 'title': 'Email', 'type': 'string'}}, 'required': ['username', 'email'], 'title': 'StringMinLengthMaxLengthContraint', 'type': 'object'}


In [168]:
from pydantic import Field
class StringRegex(BaseModel):
    username: str = Field(..., min_length = 3, max_length=30, pattern='^[a-zA-Z0-9_]+$')
    email: str = Field(..., format='email')
print(StringRegex.model_json_schema())
show(StringRegex)

{'properties': {'username': {'maxLength': 30, 'minLength': 3, 'pattern': '^[a-zA-Z0-9_]+$', 'title': 'Username', 'type': 'string'}, 'email': {'format': 'email', 'title': 'Email', 'type': 'string'}}, 'required': ['username', 'email'], 'title': 'StringRegex', 'type': 'object'}


In [169]:
form = Form(StringRegex.model_json_schema())

In [170]:
form.show()

In [171]:
form.data

FormError: Value '' does not match the specified pattern '^[a-zA-Z0-9_]+$'

In [172]:
from typing import List
class Group(BaseModel):
    ids: List[int] = Field(..., min_items = 1, max_items = 10, set = True)

print(Group.model_json_schema())
show(Group)

{'properties': {'ids': {'items': {'type': 'integer'}, 'maxItems': 10, 'minItems': 1, 'set': True, 'title': 'Ids', 'type': 'array'}}, 'required': ['ids'], 'title': 'Group', 'type': 'object'}


In [173]:
from pydantic import Field

class DictMinProperties(BaseModel):
    name: str
    settings: dict[str, int] = Field(..., min_properties = 1)

print(DictMinProperties.model_json_schema())
show(DictMinProperties)

{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'settings': {'additionalProperties': {'type': 'integer'}, 'min_properties': 1, 'title': 'Settings', 'type': 'object'}}, 'required': ['name', 'settings'], 'title': 'DictMinProperties', 'type': 'object'}


## Custom validator supported in JSON schema

In [174]:

class CustomValidationFloat(BaseModel):
    discount_type: str
    amount: float
    min_purchase: float

    @field_validator('min_purchase')
    def validate_min_purchase(cls, v, values):
        if v < 50:
            raise ValueError('minimum purchase for discount must be atleast 50')
        return v
print(CustomValidationFloat.model_json_schema())

{'properties': {'discount_type': {'title': 'Discount Type', 'type': 'string'}, 'amount': {'title': 'Amount', 'type': 'number'}, 'min_purchase': {'title': 'Min Purchase', 'type': 'number'}}, 'required': ['discount_type', 'amount', 'min_purchase'], 'title': 'CustomValidationFloat', 'type': 'object'}


## Annotations

In [175]:
class StringWithAnnotations(BaseModel):
    name: str = Field(..., title='Product Name', description='some text')
print(StringWithAnnotations.model_json_schema())
show(StringWithAnnotations)

{'properties': {'name': {'description': 'some text', 'title': 'Product Name', 'type': 'string'}}, 'required': ['name'], 'title': 'StringWithAnnotations', 'type': 'object'}


## Fixed Array Size

In [176]:
from pydantic import conlist

class FixedArraySize(BaseModel):
    fixed_items: conlist(int, min_length=1, max_length=3)

print(FixedArraySize.model_json_schema())
show(FixedArraySize)

{'properties': {'fixed_items': {'items': {'type': 'integer'}, 'maxItems': 3, 'minItems': 1, 'title': 'Fixed Items', 'type': 'array'}}, 'required': ['fixed_items'], 'title': 'FixedArraySize', 'type': 'object'}


## Custom Validators / Complex Constraints

In [177]:
class CustomValidationModel(BaseModel):
    username: str
    email: str

    @field_validator('email')
    def validate_call(cls,v):
        if not v.endswith('@example.com'):
            raise ValueError('Wrong domain')
        return v
    

print(CustomValidationModel.model_json_schema())

{'properties': {'username': {'title': 'Username', 'type': 'string'}, 'email': {'title': 'Email', 'type': 'string'}}, 'required': ['username', 'email'], 'title': 'CustomValidationModel', 'type': 'object'}


# Dynamic fields

In [178]:
from datetime import datetime

class DynamicFields(BaseModel):
    timestamp: datetime = Field(default_factory = datetime.utcnow)


print(Event.model_json_schema())
show(DynamicFields)

NameError: name 'Event' is not defined

## Aliases

In [179]:
class IntAlias(BaseModel):
    user_id: int = Field(..., alias = 'id' )

print(IntAlias.model_json_schema())
show(IntAlias)

{'properties': {'id': {'title': 'Id', 'type': 'integer'}}, 'required': ['id'], 'title': 'IntAlias', 'type': 'object'}


In [180]:
class IntField(BaseModel):
    user_id: int = Field(...)

print(IntField.model_json_schema())
show(IntField)

{'properties': {'user_id': {'title': 'User Id', 'type': 'integer'}}, 'required': ['user_id'], 'title': 'IntField', 'type': 'object'}


## Private fields

In [181]:
from pydantic import PrivateAttr

class StringPrivateField(BaseModel):
    _password: str = PrivateAttr()
    _private_password: str
    def __init__(self, password : str):
        self._password = password

print(StringPrivateField.model_json_schema())
show(StringPrivateField)

{'properties': {}, 'title': 'StringPrivateField', 'type': 'object'}


VBox(layout=Layout(width='100%'))

## Inheritance

In [182]:
class Animal(BaseModel):
    name: str

class Dog(Animal):
    breed: str


print(Dog.model_json_schema())
show(Dog)

{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'breed': {'title': 'Breed', 'type': 'string'}}, 'required': ['name', 'breed'], 'title': 'Dog', 'type': 'object'}


In [183]:
from typing import List

class Category(BaseModel):
    name: str
    subcategories: List['Category']
print(Category.model_json_schema())
show(Category)

{'$defs': {'Category': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'subcategories': {'items': {'$ref': '#/$defs/Category'}, 'title': 'Subcategories', 'type': 'array'}}, 'required': ['name', 'subcategories'], 'title': 'Category', 'type': 'object'}}, '$ref': '#/$defs/Category'}


AttributeError: 'Form' object has no attribute '_construct_ref'

In [184]:
JSONContainer(Category.model_json_schema())

## Literals

In [185]:
from typing import Literal
class LiteralModel(BaseModel):
    category: Literal['clothing' , 'food' ]
print(LiteralModel.model_json_schema())
show(LiteralModel)

{'properties': {'category': {'enum': ['clothing', 'food'], 'title': 'Category', 'type': 'string'}}, 'required': ['category'], 'title': 'LiteralModel', 'type': 'object'}


In [186]:
from pydantic import conint

class Item(BaseModel):
    quantity: conint(ge=0)
print(Item.model_json_schema())
show(Item)

{'properties': {'quantity': {'minimum': 0, 'title': 'Quantity', 'type': 'integer'}}, 'required': ['quantity'], 'title': 'Item', 'type': 'object'}


## Enums

In [187]:
from enum import Enum
from typing import List

class Color(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class ListEnum(BaseModel):
    colors: List[Color]

print(ListEnum.model_json_schema())
show(ListEnum)

{'$defs': {'Color': {'enum': ['red', 'blue', 'green'], 'title': 'Color', 'type': 'string'}}, 'properties': {'colors': {'items': {'$ref': '#/$defs/Color'}, 'title': 'Colors', 'type': 'array'}}, 'required': ['colors'], 'title': 'ListEnum', 'type': 'object'}


## Annotated

In [188]:
from typing import Annotated

class Modelannotated(BaseModel):
    username: Annotated[str, 'Must be 12 characters', 'Must be alphanumeric']

print(Modelannotated.model_json_schema())
show(Modelannotated)

{'properties': {'username': {'title': 'Username', 'type': 'string'}}, 'required': ['username'], 'title': 'Modelannotated', 'type': 'object'}


## Complex types

In [189]:
from uuid import UUID
class UUIDModel(BaseModel):
    event_id: UUID
print(UUIDModel.model_json_schema())

{'properties': {'event_id': {'format': 'uuid', 'title': 'Event Id', 'type': 'string'}}, 'required': ['event_id'], 'title': 'UUIDModel', 'type': 'object'}


## Union

In [198]:
from typing import Union

class Item(BaseModel):
    price: Union[int, float]
    
class UnionIntFloat(BaseModel):
    price: int | float
    
print(UnionIntFloat.model_json_schema())
show(UnionIntFloat)

demo
{'properties': {'price': {'anyOf': [{'type': 'integer'}, {'type': 'number'}], 'title': 'Price'}}, 'required': ['price'], 'title': 'UnionIntFloat', 'type': 'object'}


In [199]:
from typing import Union, Dict
class DictUnion(BaseModel):
    settings: Dict[str, int]
print(DictUnion.model_json_schema())
show(DictUnion)

{'properties': {'settings': {'additionalProperties': {'type': 'integer'}, 'title': 'Settings', 'type': 'object'}}, 'required': ['settings'], 'title': 'DictUnion', 'type': 'object'}


VBox(layout=Layout(width='100%'))

In [200]:
from typing import Union, List, Dict

class DictOrListModel(BaseModel):
    data: Union[str, List[int]]
print(DictOrListModel.model_json_schema())
show(DictOrListModel)

{'properties': {'data': {'anyOf': [{'type': 'string'}, {'items': {'type': 'integer'}, 'type': 'array'}], 'title': 'Data'}}, 'required': ['data'], 'title': 'DictOrListModel', 'type': 'object'}


In [201]:
from typing import Union

class UserA(BaseModel):
    name: str
    age: int

class UserB(BaseModel):
    username: str
    email: str

class UnionModel(BaseModel):
    user: Union[UserA, UserB]

print(UnionModel.model_json_schema())
show(UnionModel)

{'$defs': {'UserA': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}}, 'required': ['name', 'age'], 'title': 'UserA', 'type': 'object'}, 'UserB': {'properties': {'username': {'title': 'Username', 'type': 'string'}, 'email': {'title': 'Email', 'type': 'string'}}, 'required': ['username', 'email'], 'title': 'UserB', 'type': 'object'}}, 'properties': {'user': {'anyOf': [{'$ref': '#/$defs/UserA'}, {'$ref': '#/$defs/UserB'}], 'title': 'User'}}, 'required': ['user'], 'title': 'UnionModel', 'type': 'object'}


In [202]:
from typing import Union
from uuid import UUID

from pydantic import BaseModel


class User(BaseModel):
    id: Union[int, str, UUID]
    name: str

print(User.model_json_schema())
show(User)

user_01 = User(id=123, name='John Doe')
print(user_01)
#> id=123 name='John Doe'
print(user_01.id)
#> 123
user_02 = User(id='1234', name='John Doe')
print(user_02)
#> id='1234' name='John Doe'
print(user_02.id)
#> 1234
user_03_uuid = UUID('cf57432e-809e-4353-adbd-9d5c0d733868')
user_03 = User(id=user_03_uuid, name='John Doe')
print(user_03)
#> id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
print(user_03.id)
#> cf57432e-809e-4353-adbd-9d5c0d733868
print(user_03_uuid.int)
#> 275603287559914445491632874575877060712

{'properties': {'id': {'anyOf': [{'type': 'integer'}, {'type': 'string'}, {'format': 'uuid', 'type': 'string'}], 'title': 'Id'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['id', 'name'], 'title': 'User', 'type': 'object'}


id=123 name='John Doe'
123
id='1234' name='John Doe'
1234
id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
cf57432e-809e-4353-adbd-9d5c0d733868
275603287559914445491632874575877060712


In [203]:
from typing import Literal, Union

from pydantic import BaseModel, Field, ValidationError


class Cat(BaseModel):
    pet_type: Literal['cat']
    meows: int


class Dog(BaseModel):
    pet_type: Literal['dog']
    barks: float


class Lizard(BaseModel):
    pet_type: Literal['reptile', 'lizard']
    scales: bool


class AnimalModel(BaseModel):
    pet: Union[Cat, Dog, Lizard] = Field(discriminator='pet_type')
    n: int

print(AnimalModel.model_json_schema())
show(AnimalModel)


print(AnimalModel(pet={'pet_type': 'dog', 'barks': 3.14}, n=1))
#> pet=Dog(pet_type='dog', barks=3.14) n=1
try:
    AnimalModel(pet={'pet_type': 'dog'}, n=1)
except ValidationError as e:
    print(e)
    """
    1 validation error for AnimalModel
    pet.dog.barks
      Field required [type=missing, input_value={'pet_type': 'dog'}, input_type=dict]
    """

{'$defs': {'Cat': {'properties': {'pet_type': {'const': 'cat', 'title': 'Pet Type', 'type': 'string'}, 'meows': {'title': 'Meows', 'type': 'integer'}}, 'required': ['pet_type', 'meows'], 'title': 'Cat', 'type': 'object'}, 'Dog': {'properties': {'pet_type': {'const': 'dog', 'title': 'Pet Type', 'type': 'string'}, 'barks': {'title': 'Barks', 'type': 'number'}}, 'required': ['pet_type', 'barks'], 'title': 'Dog', 'type': 'object'}, 'Lizard': {'properties': {'pet_type': {'enum': ['reptile', 'lizard'], 'title': 'Pet Type', 'type': 'string'}, 'scales': {'title': 'Scales', 'type': 'boolean'}}, 'required': ['pet_type', 'scales'], 'title': 'Lizard', 'type': 'object'}}, 'properties': {'pet': {'discriminator': {'mapping': {'cat': '#/$defs/Cat', 'dog': '#/$defs/Dog', 'lizard': '#/$defs/Lizard', 'reptile': '#/$defs/Lizard'}, 'propertyName': 'pet_type'}, 'oneOf': [{'$ref': '#/$defs/Cat'}, {'$ref': '#/$defs/Dog'}, {'$ref': '#/$defs/Lizard'}], 'title': 'Pet'}, 'n': {'title': 'N', 'type': 'integer'}}, '

pet=Dog(pet_type='dog', barks=3.14) n=1
1 validation error for AnimalModel
pet.dog.barks
  Field required [type=missing, input_value={'pet_type': 'dog'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [204]:
from typing import Literal, Union

from typing_extensions import Annotated

from pydantic import BaseModel, Field, ValidationError


class BlackCat(BaseModel):
    pet_type: Literal['cat']
    color: Literal['black']
    black_name: str


class WhiteCat(BaseModel):
    pet_type: Literal['cat']
    color: Literal['white']
    white_name: str


Cat = Annotated[Union[BlackCat, WhiteCat], Field(discriminator='color')]


class Dog(BaseModel):
    pet_type: Literal['dog']
    name: str


Pet = Annotated[Union[Cat, Dog], Field(discriminator='pet_type')]


class Model(BaseModel):
    pet: Pet
    n: int

print(Model.model_json_schema())
show(Model)

m = Model(pet={'pet_type': 'cat', 'color': 'black', 'black_name': 'felix'}, n=1)
print(m)
#> pet=BlackCat(pet_type='cat', color='black', black_name='felix') n=1
try:
    Model(pet={'pet_type': 'cat', 'color': 'red'}, n='1')
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    pet.cat
      Input tag 'red' found using 'color' does not match any of the expected tags: 'black', 'white' [type=union_tag_invalid, input_value={'pet_type': 'cat', 'color': 'red'}, input_type=dict]
    """
try:
    Model(pet={'pet_type': 'cat', 'color': 'black'}, n='1')
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    pet.cat.black.black_name
      Field required [type=missing, input_value={'pet_type': 'cat', 'color': 'black'}, input_type=dict]
    """

{'$defs': {'BlackCat': {'properties': {'pet_type': {'const': 'cat', 'title': 'Pet Type', 'type': 'string'}, 'color': {'const': 'black', 'title': 'Color', 'type': 'string'}, 'black_name': {'title': 'Black Name', 'type': 'string'}}, 'required': ['pet_type', 'color', 'black_name'], 'title': 'BlackCat', 'type': 'object'}, 'Dog': {'properties': {'pet_type': {'const': 'dog', 'title': 'Pet Type', 'type': 'string'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['pet_type', 'name'], 'title': 'Dog', 'type': 'object'}, 'WhiteCat': {'properties': {'pet_type': {'const': 'cat', 'title': 'Pet Type', 'type': 'string'}, 'color': {'const': 'white', 'title': 'Color', 'type': 'string'}, 'white_name': {'title': 'White Name', 'type': 'string'}}, 'required': ['pet_type', 'color', 'white_name'], 'title': 'WhiteCat', 'type': 'object'}}, 'properties': {'pet': {'discriminator': {'mapping': {'cat': {'discriminator': {'mapping': {'black': '#/$defs/BlackCat', 'white': '#/$defs/WhiteCat'}, 'propertyName

pet=BlackCat(pet_type='cat', color='black', black_name='felix') n=1
1 validation error for Model
pet.cat
  Input tag 'red' found using 'color' does not match any of the expected tags: 'black', 'white' [type=union_tag_invalid, input_value={'pet_type': 'cat', 'color': 'red'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/union_tag_invalid
1 validation error for Model
pet.cat.black.black_name
  Field required [type=missing, input_value={'pet_type': 'cat', 'color': 'black'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


## Callables

In [205]:
from typing import Any, Literal, Union

from typing_extensions import Annotated

from pydantic import BaseModel, Discriminator, Tag


class Pie(BaseModel):
    time_to_cook: int
    num_ingredients: int


class ApplePie(Pie):
    fruit: Literal['apple'] = 'apple'


class PumpkinPie(Pie):
    filling: Literal['pumpkin'] = 'pumpkin'


def get_discriminator_value(v: Any) -> str:
    if isinstance(v, dict):
        return v.get('fruit', v.get('filling'))
    return getattr(v, 'fruit', getattr(v, 'filling', None))


class ThanksgivingDinner(BaseModel):
    dessert: Annotated[
        Union[
            Annotated[ApplePie, Tag('apple')],
            Annotated[PumpkinPie, Tag('pumpkin')],
        ],
        Discriminator(get_discriminator_value),
    ]

print(ThanksgivingDinner.model_json_schema())
show(ThanksgivingDinner)


apple_variation = ThanksgivingDinner.model_validate(
    {'dessert': {'fruit': 'apple', 'time_to_cook': 60, 'num_ingredients': 8}}
)
print(repr(apple_variation))
"""
ThanksgivingDinner(dessert=ApplePie(time_to_cook=60, num_ingredients=8, fruit='apple'))
"""

pumpkin_variation = ThanksgivingDinner.model_validate(
    {
        'dessert': {
            'filling': 'pumpkin',
            'time_to_cook': 40,
            'num_ingredients': 6,
        }
    }
)
print(repr(pumpkin_variation))
"""
ThanksgivingDinner(dessert=PumpkinPie(time_to_cook=40, num_ingredients=6, filling='pumpkin'))
"""

{'$defs': {'ApplePie': {'properties': {'time_to_cook': {'title': 'Time To Cook', 'type': 'integer'}, 'num_ingredients': {'title': 'Num Ingredients', 'type': 'integer'}, 'fruit': {'const': 'apple', 'default': 'apple', 'title': 'Fruit', 'type': 'string'}}, 'required': ['time_to_cook', 'num_ingredients'], 'title': 'ApplePie', 'type': 'object'}, 'PumpkinPie': {'properties': {'time_to_cook': {'title': 'Time To Cook', 'type': 'integer'}, 'num_ingredients': {'title': 'Num Ingredients', 'type': 'integer'}, 'filling': {'const': 'pumpkin', 'default': 'pumpkin', 'title': 'Filling', 'type': 'string'}}, 'required': ['time_to_cook', 'num_ingredients'], 'title': 'PumpkinPie', 'type': 'object'}}, 'properties': {'dessert': {'oneOf': [{'$ref': '#/$defs/ApplePie'}, {'$ref': '#/$defs/PumpkinPie'}], 'title': 'Dessert'}}, 'required': ['dessert'], 'title': 'ThanksgivingDinner', 'type': 'object'}


ThanksgivingDinner(dessert=ApplePie(time_to_cook=60, num_ingredients=8, fruit='apple'))
ThanksgivingDinner(dessert=PumpkinPie(time_to_cook=40, num_ingredients=6, filling='pumpkin'))


"\nThanksgivingDinner(dessert=PumpkinPie(time_to_cook=40, num_ingredients=6, filling='pumpkin'))\n"

In [40]:
from typing import Callable
class CallablesModel(BaseModel):
    process: Callable[[int],int]

print(CallablesModel.model_json_schema())

PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.CallableSchema

For further information visit https://errors.pydantic.dev/2.10/u/invalid-for-json-schema

In [41]:
class BasicTypes(BaseModel):
    string_field: str
    int_field: int
    float_field: float
    bool_field: bool
    none_field: None

In [42]:
## Recursion

In [43]:

class RecursiveModel(BaseModel):
    value: str
    children: List['RecursiveModel'] = []

In [44]:
from pydantic import SecretStr

class StringValidations(BaseModel):
    min_length: str = Field(min_length=3)
    max_length: str = Field(max_length=10)
    pattern: str = Field(pattern=r"^[a-zA-Z0-9]+$")
    password: SecretStr

In [45]:
from pydantic import PositiveInt, NegativeInt, conint, PositiveFloat, confloat


class NumericValidations(BaseModel):
    positive_int: PositiveInt
    negative_int: NegativeInt
    range_int: conint(ge=0, le=100)
    positive_float: PositiveFloat
    range_float: confloat(ge=0.0, le=1.0)

In [46]:
## Enum

In [156]:
class UserType(str, Enum):
    ADMIN = "admin"
    USER = "user"
    GUEST = "guest"

class EnumModel(BaseModel):
    user_type: UserType
    status: Literal["active", "inactive", "pending"]
    level: int = Field(ge=1, le=5)

show(EnumModel)

FormError: Expecting type information for non-enum properties

In [48]:
## Nested Model

In [49]:
from typing import Any
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str = Field(pattern=r"^\d{5}$")

class NestedModel(BaseModel):
    id: UUID
    name: str
    addresses: List[Address]
    primary_address: Address
    metadata: Dict[str, Any]

show(NestedModel)

FormError: Expecting type information for non-enum properties

## Generics / Template

In [50]:
from typing import TypeVar, Generic

T = TypeVar('T')
class Generics(BaseModel, Generic[T]):
    data: T
    status: int
    message: str



In [51]:
JSONContainer(Generics[int].model_json_schema())

## Network model

In [52]:
from pydantic import BaseModel
from pydantic.networks import IPvAnyAddress
class IpModel(BaseModel):
    ip: IPvAnyAddress

JSONContainer(IpModel.model_json_schema())

## Json

In [53]:
from pydantic import Json
class DynamicConfig(BaseModel):
    raw_json: Json
    metadata: Dict[str, Any]
    settings_json: Json[Dict[str, Any]]

show(DynamicConfig)

KeyError: 'properties'

In [54]:
JSONContainer(DynamicConfig.model_json_schema())

## Optional typing

In [55]:
from typing import Optional
class OptionalModel(BaseModel):
    field1: Optional[str] = None
    field2: Optional[int] = None
    field3: Optional[List[str]] = None


show(OptionalModel)

FormError: Schemas within anyOf/oneOf/allOf need to set the title field

In [56]:
JSONContainer(OptionalModel.model_json_schema())

In [57]:
import json
import logging
import pydantic
from typing import List, Type
from pathlib import Path

def generate_json_schema_files(models: List[Type[BaseModel]], output_dir: str = 'schemas'):
    """
    Generate JSON schema files for a list of pydantic basemodels
    """
    schema_dir = Path(output_dir)
    schema_dir.mkdir(parents=True, exist_ok=True)

    for model in models:
        try:
            model_name = model.__name__.lower()
            schema = model.model_json_schema()
            schema_path = schema_dir / f'{model_name}.json'

            with open(schema_path, 'w', encoding='utf8') as f:
                json.dump(schema,f, indent=2, ensure_ascii=False)

            logging.info(f'Generated schema for {model_name} at {schema_path}')

        except Exception as e:
            logging.error(f'Failed to generate schema for {model.__name__}: {str(e)}')
            





models_to_process = [StringSimple, StringNested, StringAndInt, FloatMultipleOfMinumumContraint, StringMinLengthMaxLengthContraint, StringRegex, Group, DictMinProperties, CustomValidationFloat, StringWithAnnotations,
                     FixedArraySize, CustomValidationModel, DynamicFields, IntAlias, IntField, StringPrivateField, Dog, Category, LiteralModel, Item, ListEnum, Modelannotated, UUIDModel, UnionIntFloat,DictUnion,
                    BasicTypes, RecursiveModel, StringValidations, NumericValidations, EnumModel,NestedModel,Generics, IpModel, DynamicConfig, OptionalModel ]
user_generated_models = []
generate_json_schema_files(models_to_process, 'schemas')
generate_json_schema_files(user_generated_models, 'schemas-user')

In [58]:
from ipywidgets_jsonschema import Form


#schema = Config.model_json_schema()

form = Form(ListEnum.model_json_schema())
form.show(width='500px')

In [59]:
form.data

{'colors': []}

## Unsupported so far
Enums (cant add entries),
Union

## Unsupported by JSON schema
Custom validators, dynamic computed fields